In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
X = train.drop('class', axis=1)
y = train['class']
X_test = test

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=test.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=test.columns)

C:\Users\hungu\anaconda.x\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\hungu\anaconda.x\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\hungu\anaconda.x\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=101)

model.fit(x_train, y_train)

pred_y = model.predict_proba(x_val)

C:\Users\hungu\anaconda.x\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [6]:
from sklearn.metrics import log_loss, accuracy_score
log_loss(y_val, pred_y)

0.48659927130885877

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 10)

In [8]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   50.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   57.0s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=101, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110], 'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 200, 300, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [9]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 100}

In [10]:
err = []
y_pred_tot_rf = []
features = [col for col in X.columns if col not in ["class"]]

from sklearn.model_selection import StratifiedKFold, KFold
feature_importance_df = pd.DataFrame()

fold = StratifiedKFold(n_splits=20, shuffle=True)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = RandomForestClassifier(random_state=101, 
                               max_depth=14, 
                               n_estimators=100,
                               max_features=5, 
                               min_samples_leaf=4, 
                               min_samples_split=10)
    m.fit(x_train, y_train)
    pred_y = m.predict_proba(x_val)
    print(i, "err_rf: ",log_loss(y_val,pred_y))
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_rf.append(pred_test)

1 err_rf:  0.29051705681662304
2 err_rf:  0.33565480982377
3 err_rf:  0.2733566255427439
4 err_rf:  0.29965259592028853
5 err_rf:  0.2659786853895329
6 err_rf:  0.2718315343899585
7 err_rf:  0.32057062008991694
8 err_rf:  0.2653821243059303
9 err_rf:  0.2938860856764548
10 err_rf:  0.40269728552467143
11 err_rf:  0.27368675309712504
12 err_rf:  0.27846463278024813
13 err_rf:  0.24069633097806692
14 err_rf:  0.28043431443400446
15 err_rf:  0.2949724699800946
16 err_rf:  0.3736332927289014
17 err_rf:  0.35816658036930377
18 err_rf:  0.33216771200840967
19 err_rf:  0.3526668448628008
20 err_rf:  0.320985403376823


In [11]:
np.mean(err,0)

0.3062700879047834

In [12]:
all_feat = feature_importance_df[["Feature", 
                                      "importance"]].groupby("Feature").mean().sort_values(by="importance", 
                                                                                           ascending=False)
all_feat.reset_index(inplace=True)
important_feat = list(all_feat['Feature'])
all_feat

,Feature,importance
0,thickness,0.152570
1,x_component_3,0.122893
2,x_component_1,0.109949
3,x_component_2,0.100230
4,pixel_area,0.063763
5,max_luminosity,0.061683
6,log_area,0.058591
7,ymax,0.057800
8,xmin,0.055308
9,ymin,0.053672


In [13]:
df = X[important_feat]
corr_matrix = df.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

high_cor = [column for column in upper.columns if any(upper[column] > 0.98)]
print(len(high_cor))
print(high_cor)

4
['log_area', 'ymin', 'xmax', 'grade_A_Component_1']


In [14]:
features = [i for i in important_feat if i not in high_cor]
print(len(features))
print(features)

11
['thickness', 'x_component_3', 'x_component_1', 'x_component_2', 'pixel_area', 'max_luminosity', 'ymax', 'xmin', 'x_component_4', 'x_component_5', 'grade_A_Component_2']


In [15]:
X1 = X[features]
X_test1 = X_test[features]

In [16]:
err = []
y_pred_tot_rf = []

from sklearn.model_selection import StratifiedKFold, KFold

fold = StratifiedKFold(n_splits=20, shuffle=True)
i = 1
for train_index, test_index in fold.split(X1, y):
    x_train, x_val = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = RandomForestClassifier(random_state=101, 
                               max_depth=14, 
                               n_estimators=100,
                               max_features=5, 
                               min_samples_leaf=4, 
                               min_samples_split=10)
    m.fit(x_train, y_train)
    pred_y = m.predict_proba(x_val)
    print(i, "err_rf: ",log_loss(y_val,pred_y))
    
    
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test1)
    i = i + 1
    y_pred_tot_rf.append(pred_test)

1 err_rf:  0.32273478823009155
2 err_rf:  0.23505470509668303
3 err_rf:  0.2654790349690291
4 err_rf:  0.29653864069291913
5 err_rf:  0.21976938987873237
6 err_rf:  0.2916006308303248
7 err_rf:  0.33740261455361453
8 err_rf:  0.3158662438223235
9 err_rf:  0.2430956945709848
10 err_rf:  0.29552729898983027
11 err_rf:  0.3154664143069259
12 err_rf:  0.2538934823968646
13 err_rf:  0.37052473777772055
14 err_rf:  0.30298160331384066
15 err_rf:  0.23624221121372924
16 err_rf:  0.27522762297133085
17 err_rf:  0.286161109078772
18 err_rf:  0.3951802199334562
19 err_rf:  0.28347974224962585
20 err_rf:  0.35176617186295095


In [17]:
np.mean(err,0), err[17]

(0.2946996178369875, 0.3951802199334562)

In [18]:
submission = pd.DataFrame(y_pred_tot_rf[17])
submission.columns = [1,2]
submission.to_excel('Submission.xlsx', index=False)

In [19]:
submission.head()

,1,2
0,0.998932,0.001068
1,0.193785,0.806215
2,0.960268,0.039732
3,1.000000,0.000000
4,0.475845,0.524155
